In [3]:
import numpy as np
import pandas as pd
from sympy import primerange
from sympy.utilities.iterables import multiset_permutations
from multiprocessing import Pool, cpu_count
import numba
from sympy import isprime
from math import sqrt

In [4]:
cities = pd.read_csv('../data/raw/cities.csv', index_col=['CityId'], dtype={'X': np.float32, 'Y': np.float32})

In [5]:
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([not isprime(city_id) for city_id in cities.index], dtype=np.int32)

@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def pure_score(path):
    '''Pure path score without penalties.'''
    dist = 0.0
    for i in numba.prange(path.shape[0] - 1):
        a, b = XY[path[i]], XY[path[i+1]]
        dx, dy = a[0] - b[0], a[1] - b[1]
        dist += sqrt(dx * dx + dy * dy)
    return dist


@numba.jit('f8(i4, i8[:])', nopython=True, parallel=False)
def chunk_score(start_offset, chunk):
    '''Score of path's chunk that starts at index 'start_offset'.'''
    dist = 0.0
    penalty = 0.0
    penalty_modulo = 9 - start_offset % 10
    for i in numba.prange(chunk.shape[0] - 1):
        id_a = chunk[i]
        a, b = XY[id_a], XY[chunk[i+1]]
        dx, dy = a[0] - b[0], a[1] - b[1]
        d = sqrt(dx * dx + dy * dy)
        dist += d
        if i % 10 == penalty_modulo and is_not_prime[id_a]:
            penalty += d
    return dist + 0.1 * penalty


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def path_score(path):
    return chunk_score(0, path)

In [6]:
tour = pd.read_csv('submission.151557248.csv')['Path'].values
path_score(tour)

1515572.4502716302

In [7]:
def riffle2(batch, i, permutations):
    best = batch
    best_score = chunk_score(i, best)
    for p in permutations:
        perm = batch[p]
        perm_score = chunk_score(i, perm)
        if perm_score < best_score:
#             print(str(best_score) +" ---> "+str(perm_score))
            best = perm
            best_score = perm_score
    if (best != batch).any():
        return best
    else:
        return None

In [23]:
def riffle(split_tour1):
    order, batch, step, nperms = split_tour1
    n = nperms + 2
    permutations = np.array([np.array([0]+i+[nperms+1]) for i in multiset_permutations(np.arange(1, nperms+1))])
    for i in range(0, batch.size-n):
        if (i + n >= batch.size): break
        r = riffle2(batch[i:i+n], order * step + i, permutations)
        if r is not None:
#             print(r,i)
#             print(batch[i:i+n])
#             print(r)
            score_old = chunk_score(order * step, batch)
            batch_new = np.concatenate((batch[:i], r, batch[i+n:]))
            score_new = chunk_score(order * step, batch_new)
            if (score_new < score_old):
                batch = batch_new
                print(str(score_old) + " -> " + str(score_new))
    return [order, batch]

In [58]:
def multi_riffle(tour, ncores, step, nperms):
    p = Pool(ncores)
    t = [tour[i*step:(i+1)*step] for i in range(0,(tour.size//step)+1)]
    t = [(i,t,step,nperms) for i, t in enumerate(t)]
    ret = p.map(riffle, t)

    ret_d = {}
    for i in range(len(ret)):
        ret_d[ret[i][0]] = ret[i][1]
    ret2 = []
    for i in range(len(ret_d)):
        ret2 += list(ret_d[i])
    return ret2

In [59]:
# %%time
step = 100
nperms=3
ncores=2
tour_new = multi_riffle(tour, ncores, step, nperms)

CPU times: user 418 ms, sys: 103 ms, total: 521 ms
Wall time: 22 s


In [60]:
pd.DataFrame({'Path': list(tour_new)}).to_csv('submission.csv', index=False)

In [61]:
path_score(np.array(tour_new, dtype=np.int64))

1515572.4502716302